# LeNet Lab
![LeNet Architecture](lenet.png)
Source: Yan LeCun

## Load Data

Load the MNIST data, which comes pre-loaded with TensorFlow.

You do not need to modify this section.

In [ ]:
import tensorflow as tf

%store -r current_data_dict
%store -r current_data_dict_name
tf.reset_default_graph()

X_train, y_train           = current_data_dict['X_train'], current_data_dict['y_train']
X_validation, y_validation = current_data_dict['X_valid'], current_data_dict['y_valid']
X_test, y_test             = current_data_dict['X_test'], current_data_dict['y_test']
X_my_test, y_my_test       = current_data_dict['X_my_test'], current_data_dict['y_my_test']


assert(len(X_train) == len(y_train))
assert(len(X_validation) == len(y_validation))
assert(len(X_test) == len(y_test))
assert(len(X_my_test) == len(y_my_test))

# print()
# print("Image Shape: {}".format(X_train[0].shape))
# print()
# print("Training Set:   {} samples".format(len(X_train)))
# print("Validation Set: {} samples".format(len(X_validation)))
# print("Test Set:       {} samples".format(len(X_test)))

The MNIST data that TensorFlow pre-loads comes as 28x28x1 images.

However, the LeNet architecture only accepts 32x32xC images, where C is the number of color channels.

In order to reformat the MNIST data into a shape that LeNet will accept, we pad the data with two rows of zeros on the top and bottom, and two columns of zeros on the left and right (28+2+2 = 32).

You do not need to modify this section.

In [2]:
import numpy as np

# Pad images with 0s
# X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
# X_validation = np.pad(X_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
# X_test       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
# print("Updated Image Shape: {}".format(X_train[0].shape))

## Visualize Data

View a sample from the dataset.

You do not need to modify this section.

In [3]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

index = random.randint(0, len(X_train))
image = X_train[index].squeeze()
# print(image.shape)

# plt.figure(figsize=(1,1))
# plt.imshow(image, cmap="gray")
# print(y_train[index])

## Preprocess Data

Shuffle the training data.

You do not need to modify this section.

In [4]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

## Setup TensorFlow
The `EPOCH` and `BATCH_SIZE` values affect the training speed and model accuracy.

You do not need to modify this section.

In [5]:
EPOCHS = 2
BATCH_SIZE = 128

## TODO: Implement LeNet-5
Implement the [LeNet-5](http://yann.lecun.com/exdb/lenet/) neural network architecture.

This is the only cell you need to edit.
### Input
The LeNet architecture accepts a 32x32xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

### Architecture
**Layer 1: Convolutional.** The output shape should be 28x28x6.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 14x14x6.

**Layer 2: Convolutional.** The output shape should be 10x10x16.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 5x5x16.

**Flatten.** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. The easiest way to do is by using `tf.contrib.layers.flatten`, which is already imported for you.

**Layer 3: Fully Connected.** This should have 120 outputs.

**Activation.** Your choice of activation function.

**Layer 4: Fully Connected.** This should have 84 outputs.

**Activation.** Your choice of activation function.

**Layer 5: Fully Connected (Logits).** This should have 10 outputs.

### Output
Return the result of the 2nd fully connected layer.

In [6]:
from tensorflow.contrib.layers import flatten

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

def LeNet(x):    
    # TODO(saajan): Use this function
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    n_classes = 43
    dropout = 0.25  # Dropout, probability to keep units
    
    weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, x.get_shape()[3].value, 6], mean=mu, stddev=sigma), name='wc1Variable'),
    'wc2': tf.Variable(tf.random_normal([5, 5, 6, 16], mean=mu, stddev=sigma), name='wc2Variable'),
    'wd1': tf.Variable(tf.random_normal([400, 120], mean=mu, stddev=sigma), name='wd1Variable'),
    'wd2': tf.Variable(tf.random_normal([120, 84], mean=mu, stddev=sigma), name='wd2Variable'),
    'out': tf.Variable(tf.random_normal([84, n_classes], mean=mu, stddev=sigma), name='outVariable')
        
    }

    biases = {
        'bc1': tf.Variable(tf.random_normal([6]), name='bc1Variable'),
        'bc2': tf.Variable(tf.random_normal([16]), name='bc2Variable'),
        'bd1': tf.Variable(tf.random_normal([120]), name='bd1Variable'),
        'bd2': tf.Variable(tf.random_normal([84]), name='bd2Variable'),
        'out': tf.Variable(tf.random_normal([n_classes]), name='outVariable')
        
    }
    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    # TODO: Activation.
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = maxpool2d(conv1, k=2)
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    # TODO: Activation and dropout.
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = maxpool2d(conv2, k=2)

    # TODO: Flatten. Input = 5x5x16. Output = 400.
    flat = tf.contrib.layers.flatten(conv2)
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1 = tf.add(tf.matmul(flat, weights['wd1']), biases['bd1'])
    # TODO: Activation.
    fc1 = tf.nn.relu(fc1)

    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2 = tf.add(tf.matmul(fc1, weights['wd2']), biases['bd2'])
    # TODO: Activation snd dropout.
    fc2 = tf.nn.relu(fc2)

    # TODO: Layer 5: Fully Connected. Input = 84. Output = 43.
    logits = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    
    return logits

## Features and Labels
Train LeNet to classify [MNIST](http://yann.lecun.com/exdb/mnist/) data.

`x` is a placeholder for a batch of input images.
`y` is a placeholder for a batch of output labels.

You do not need to modify this section.

In [7]:
x = tf.placeholder(tf.float32, (None, 32, 32, X_train[0].shape[2]), name='xplaceholder')
y = tf.placeholder(tf.int32, (None), name='yplaceholder')
one_hot_y = tf.one_hot(y, 43)

## Training Pipeline
Create a training pipeline that uses the model to classify MNIST data.

You do not need to modify this section.

In [8]:
rate = 0.001

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

## Model Evaluation
Evaluate how well the loss and accuracy of the model for a given dataset.

You do not need to modify this section.

In [9]:
prediction_values = tf.argmax(logits, 1)
softmax = tf.nn.softmax(logits)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver(max_to_keep=100000)

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

## Train the Model
Run the training data through the training pipeline to train the model.

Before each epoch, shuffle the training set.

After each epoch, measure the loss and accuracy of the validation set.

Save the model after training.

You do not need to modify this section.

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from pathlib import Path
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)


session_name = './data/' + current_data_dict_name

num_examples = len(X_train)

train_acc_list = []
valid_acc_list = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Progress bar
    epochs_pbar = tqdm(range(EPOCHS), desc='Epoch', unit='epochs')
    for i in epochs_pbar:

        if not Path(session_name + "_" + str(i) + ".meta").is_file():
            #print("Training...")
            #print()
            X_train, y_train = shuffle(X_train, y_train)
            for offset in range(0, num_examples, BATCH_SIZE):
                end = offset + BATCH_SIZE
                batch_x, batch_y = X_train[offset:end], y_train[offset:end]                
                sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            saver.save(sess, session_name + "_" + str(i))
            #print("Model saved at: " + session_name + "_" + str(i) +  "...")
        else:
            #print("Model exists. Restoring from: " + session_name + "_" + str(i) +  "...")
            sess.run(tf.global_variables_initializer())
            saver.restore(sess, session_name + "_" + str(i))
            optimizer = tf.get_collection("optimizer")


        training_accuracy = evaluate(X_train, y_train)
        validation_accuracy = evaluate(X_validation, y_validation)

        train_acc_list.append(training_accuracy)
        valid_acc_list.append(validation_accuracy)

#         print("EPOCH {} ...".format(i+1))
#         print("Training Accuracy = {:.3f}".format(training_accuracy))
#         print("Validation Accuracy = {:.3f}".format(validation_accuracy))
#         print()

acc_plot = plt.subplot(211)
acc_plot.set_title('Accuracy')
batches = [i for i in range(1, len(train_acc_list)+1, 1)]
acc_plot.plot(batches, train_acc_list, 'r', label='Training Accuracy')
acc_plot.plot(batches, valid_acc_list, 'x', label='Validation Accuracy')
acc_plot.set_ylim([0, 1.0])
acc_plot.set_xlim([batches[0], batches[-1]])
acc_plot.legend(loc=4)
plt.tight_layout()
plt.show()

# print("Training accuracies: ")
# print(train_acc_list)
# print("Validation accuracies: ")
# print(valid_acc_list)

# saver.save(sess, session_name)
# print("Model saved at: " + session_name +  "...")
        
# else:
#     print("Model already saved. Restoring from: " + session_name + "...")

## Evaluate the Model
Once you are completely satisfied with your model, evaluate the performance of the model on the test set.

Be sure to only do this once!

If you were to measure the performance of your trained model on the test set, then improve your model, and then measure the performance of your model on the test set again, that would invalidate your test results. You wouldn't get a true measure of how well your model would perform against real data.

You do not need to modify this section.

In [10]:
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    
    saver.restore(sess, session_name + "_" + str(EPOCHS-1))
    optimizer = tf.get_collection("optimizer")

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

NameError: name 'session_name' is not defined

In [12]:
from itertools import islice
from prettytable import PrettyTable
from texttable import Texttable

session_name = './data/' + current_data_dict_name
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    
    saver.restore(sess, session_name + "_" + str(EPOCHS-1))
    optimizer = tf.get_collection("optimizer")

    show_data([current_data_dict], 'my_test')
    
    test_accuracy = evaluate(X_my_test, y_my_test)
    print("5 images test data Accuracy = {:.3f}".format(test_accuracy))
    predictions = sess.run(prediction_values, feed_dict={x: X_my_test, y: y_my_test})
    
    #Top 5 softmax:
    softmax_output = sess.run(softmax, feed_dict={x: X_my_test, y: y_my_test})
    # Should get top 5 tuples of name -> prob
    five_name_prob_tuples_list = []
    for image_softmaxes in softmax_output:
        # 43 per image
        # Dict of 43 name keys to prob for this image
        sign_prob_dict = {}
        for idx, prob in enumerate(image_softmaxes):
            sign_prob_dict[signnames_dict[str(idx)]] = prob
        
        # sorted (by prob) 43 tuples of name to prob
        sign_prob_dict_list = sorted(sign_prob_dict.items(), key=lambda x:x[1])
        
        # top five name prob tuples for this image
        top_five_name_prob_tuples = []
        for sign,prob in sign_prob_dict_list[-1:-6:-1]:
            #top 5 name to prob tuples
            top_five_name_prob_tuples.append((sign, prob))
        five_name_prob_tuples_list.append(top_five_name_prob_tuples)   
    
    
    table_data = Texttable()
    table_rows = [['Actual value', 'Predicted value', 'Other labels\' probability']]
    #table_data = PrettyTable(['Actual value', 'Predicted value', 'Other labels\' probability'])    
    #print('Actual values | Predicted value | Competing labels prob'),
    for y1, y2, fnpts in zip(y_my_test, predictions, five_name_prob_tuples_list):
        table_rows.append([signnames_dict[str(y1)], signnames_dict[str(y2)], str([str(tup[0] + ": " + str(tup[1])) for tup in fnpts])])
#         for fnpts_item in fnpts[1:]:
#             table_rows.append(['', '', str(fnpts_item[0])[:14] + ": " + str(fnpts_item[1])[:6]])
        #print(signnames_dict[str(y1)] + " | " + signnames_dict[str(y2)] + " | " + str(fnpts))
    table_data.add_rows(table_rows)
    print (table_data.draw())
    #print(table_data)

        
#     >>> from texttable import Texttable
# >>> t = Texttable()
# >>> t.add_rows([['Name', 'Age'], ['Alice', 24], ['Bob', 19]])
# >>> print t.draw()
        
    #top_five_softmax = 
    

INFO:tensorflow:Restoring parameters from ./data/cropped_grayscaled_normalized_data_1
My data test Accuracy = 0.500
